In [1]:
import pandas as pd, skill_cat, multiprocessing.dummy
from jobs_skills_weights import *
import synonyms

In [2]:
skill_cat._refresh()
synonyms._refresh()

# What is the goal here?
For each job, create a dict from (a skill name that job wants) to (a float indicating the importance of that skill)

In [3]:
# What fraction of skill weight comes from being listed in a job description?
# 1-list_weight gives the weight of being in a category
list_weight = 4/5.0 

In [4]:
jobs = get_jobs(bookmarked=False)
raw_job_details = get_raw_job_details(jobs)

In [5]:
skill_cat._refresh()
synonyms._refresh()

In [14]:
# Construct dataframe from raw job details, 
#   summing between skills listed under multiple teal categories, 
#   and filtered by skills I have categories for (a loose standin for skills present on the resume).
job_skill_text_counts = pd.concat(
    {key: extract_skills_data(value) for key, value in raw_job_details.items()},
    names=["id"]
).groupby(level=[0,2,3]).sum().query("`skill`.isin(@skill_cat.skill_to_categories.keys())").sort_index()

In [62]:
job_skill_text_counts

count
id                                   skill                skill text                 
0124e218-2d33-41e8-907b-5228ea386455 automation           automating                1
                                     communication        communication             1
                                                          communication skills      1
                                     computer science     computer science          1
                                     containerization     devops                    2
...                                                                               ...
fdd15faa-0804-4b07-9f55-715436213e75 software engineering software engineering      2
                                     strategy             strategy                  1
                                     teamwork             teamwork                  1
                                     tensorflow           tensorflow                1
                                     testing              testing                   1

[2281 rows x 1 columns]

## Investingating the distribution of synonyms

In [7]:
job_skill_text_counts

count
id                                   skill                skill text                 
0124e218-2d33-41e8-907b-5228ea386455 automation           automating                1
                                     communication        communication             1
                                                          communication skills      1
                                     computer science     computer science          1
                                     containerization     devops                    2
...                                                                               ...
fdd15faa-0804-4b07-9f55-715436213e75 software engineering software engineering      2
                                     strategy             strategy                  1
                                     teamwork             teamwork                  1
                                     tensorflow           tensorflow                1
                                     testing              testing                   1

[2281 rows x 1 columns]

In [9]:
num_synonyms = job_skill_text_counts.reset_index().set_index(["id", "skill"])["skill text"].groupby(level=[0,1]).nunique()

In [10]:
num_synonyms

id                                    skill               
0124e218-2d33-41e8-907b-5228ea386455  automation              1
                                      communication           2
                                      computer science        1
                                      containerization        3
                                      data                    1
                                                             ..
fdd15faa-0804-4b07-9f55-715436213e75  software engineering    1
                                      strategy                1
                                      teamwork                1
                                      tensorflow              1
                                      testing                 1
Name: skill text, Length: 1947, dtype: int64

In [11]:
# What skills have the highest mean number of synonyms within a job listing?
num_synonyms.groupby(level=1).mean().sort_values(ascending=False).head(20)

skill
containerization         2.194030
algebra                  2.000000
communication            1.959184
machine learning         1.760417
scale                    1.659091
network architectures    1.500000
analysis                 1.454545
collaboration            1.333333
optimization             1.277778
web development          1.250000
implementation           1.226415
agile                    1.222222
data pipelines           1.210526
azure databricks         1.200000
project management       1.185185
managing requirements    1.125000
research                 1.120000
prototyping              1.111111
remote work              1.111111
automation               1.052632
Name: skill text, dtype: float64

In [12]:
# What is the average number of synonyms for a skill in a job listing?
num_synonyms.mean()

1.1715459681561375

## Computing `skill weights`

In [65]:
job_skills_data = job_skill_text_counts.groupby(level=[0,1]).agg(**{
    "share of skill":("count",lambda jstc: {key[2]:count/jstc.sum() for key, count in jstc.items()}),
    #"share of skill":("count",lambda jstc: [(key[2],count/jstc.sum()) for key, count in jstc.items()]),
    "count":("count","sum")
})

In [66]:
job_skills_data

share of skill  \
id                                   skill                                                                     
0124e218-2d33-41e8-907b-5228ea386455 automation                                          {'automating': 1.0}   
                                     communication         {'communication': 0.5, 'communication skills':...   
                                     computer science                              {'computer science': 1.0}   
                                     containerization      {'devops': 0.5, 'docker': 0.25, 'kubernetes': ...   
                                     data                                                      {'data': 1.0}   
...                                                                                                      ...   
fdd15faa-0804-4b07-9f55-715436213e75 software engineering                      {'software engineering': 1.0}   
                                     strategy                                              {'strategy': 1.0}   
                                     teamwork                                              {'teamwork': 1.0}   
                                     tensorflow                                          {'tensorflow': 1.0}   
                                     testing                                                {'testing': 1.0}   

                                                           count  
id                                   skill                        
0124e218-2d33-41e8-907b-5228ea386455 automation                1  
                                     communication             2  
                                     computer science          1  
                                     containerization          4  
                                     data                      3  
...                                                          ...  
fdd15faa-0804-4b07-9f55-715436213e75 software engineering      2  
                                     strategy                  1  
                                     teamwork                  1  
                                     tensorflow                1  
                                     testing                   1  

[1947 rows x 2 columns]

In [70]:
jobs_categories_skills_data = pd.DataFrame([{
        "id":job_id,
        "category":category,
        "skill":skill_name,
        "count":job_skills_data["count"].get((job_id, skill_name), 0)/float(len(categories_skill_is_in)),
        "share of skill":job_skills_data["share of skill"].get((job_id, skill_name), {skill_name:1.0})
        #"share of skill":job_skills_data["share of skill"].get((job_id, skill_name), [(skill_name,1.0)])
    } for job_id in raw_job_details.keys()
        for skill_name, categories_skill_is_in in skill_cat.skill_to_categories.items()
            for category in categories_skill_is_in
]).set_index(["id", "category", "skill"]).sort_index() # Sort the index so it's easy to read. Not a requirement.

In [71]:
jobs_categories_skills_data

count  \
id                                   category skill                     
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science    0.5   
                                              experimentation     0.0   
                                              research            0.0   
                                     Admin    automation          1.0   
                                              aws                 0.0   
...                                                               ...   
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix                0.0   
                                     Visual   graphic design      0.0   
                                              usability           0.0   
                                     WebDev   javascript          0.0   
                                              web development     0.0   

                                                                           share of skill  
id                                   category skill                                        
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science  {'computer science': 1.0}  
                                              experimentation    {'experimentation': 1.0}  
                                              research                  {'research': 1.0}  
                                     Admin    automation              {'automating': 1.0}  
                                              aws                            {'aws': 1.0}  
...                                                                                   ...  
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix                          {'unix': 1.0}  
                                     Visual   graphic design      {'graphic design': 1.0}  
                                              usability                {'usability': 1.0}  
                                     WebDev   javascript              {'javascript': 1.0}  
                                              web development    {'web development': 1.0}  

[12700 rows x 2 columns]

In [72]:
counts = jobs_categories_skills_data.pop("count")
jobs_categories_skills_data["count"] = counts*list_weight + (counts*(1-list_weight)).groupby(level=[0,1]).mean()

In [73]:
jobs_categories_skills_data

share of skill  \
id                                   category skill                                         
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science  {'computer science': 1.0}   
                                              experimentation    {'experimentation': 1.0}   
                                              research                  {'research': 1.0}   
                                     Admin    automation              {'automating': 1.0}   
                                              aws                            {'aws': 1.0}   
...                                                                                   ...   
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix                          {'unix': 1.0}   
                                     Visual   graphic design      {'graphic design': 1.0}   
                                              usability                {'usability': 1.0}   
                                     WebDev   javascript              {'javascript': 1.0}   
                                              web development    {'web development': 1.0}   

                                                                   count  
id                                   category skill                       
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science  0.433333  
                                              experimentation   0.033333  
                                              research          0.033333  
                                     Admin    automation        0.870588  
                                              aws               0.070588  
...                                                                  ...  
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix              0.033333  
                                     Visual   graphic design    0.000000  
                                              usability         0.000000  
                                     WebDev   javascript        0.000000  
                                              web development   0.000000  

[12700 rows x 2 columns]

In [74]:
jobs_categories_skills_data.explode("share of skill")

share of skill  \
id                                   category skill                                
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science  computer science   
                                              experimentation    experimentation   
                                              research                  research   
                                     Admin    automation              automating   
                                              aws                            aws   
...                                                                          ...   
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix                          unix   
                                     Visual   graphic design      graphic design   
                                              usability                usability   
                                     WebDev   javascript              javascript   
                                              web development    web development   

                                                                   count  
id                                   category skill                       
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science  0.433333  
                                              experimentation   0.033333  
                                              research          0.033333  
                                     Admin    automation        0.870588  
                                              aws               0.070588  
...                                                                  ...  
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix              0.033333  
                                     Visual   graphic design    0.000000  
                                              usability         0.000000  
                                     WebDev   javascript        0.000000  
                                              web development   0.000000  

[13156 rows x 2 columns]